# 0. Download, Bereinigung und Anreicherung der zugrundeliegenden Hageldaten

In diesem Notebook werden Daten zur Anzahl der Hageltage sowie die maximal zu erwartende Hagelkorngrösse in den Sommermonaten (April-September) von 2002 bis 2022 in der Schweiz heruntergeladen und vorverarbeitet. Es handelt sich dabei um Monatsdaten pro km^2, also Gitterdaten über der Schweiz.

Da Hagel und Hagelkorngrössen nicht flächendeckend am Boden gemessen werden können, wird ein Hageltag und die Grösse aus Radarmessungen abgeleitet. 

Weitere Informationen befinden sich in https://www.nccs.admin.ch/nccs/en/home/data-and-media-library/data/hail-climate-datasets.html


Das Notebook beinhaltet dabei die folgenden Abschnitte:

* [0.1 Download der Daten](#download)

* [0.2 Bereinigung der Daten](#cleaning)

* [0.3 Anreicherung der Daten](#enrichment)

* [0.4 Speichern der Daten](#dump)

<a id="download"></a>
## 0.1 Herunterladen der Daten

In [1]:
import os
from pathlib import Path
import zipfile

### 0.1.1 Defintion einiger nützlicher Helferfunktionen

In [2]:
  def unzip_data(in_file: Path, out_path: Path) -> None:
    """Unzips Zip file content into specified output path.
    
    Args:
        in_file: The zip archive to unpack.
        out_path: The output path where to unzip the Zip archive content into.
    """
    with zipfile.ZipFile(in_file, "r") as zip_ref:
        zip_ref.extractall(out_path)
        

def create_directory(directory: Path) -> None:
    """Creates a directory if it doesn't already exists.
    
    Args:
        directory: The directory to create.
    """
    directory.mkdir(parents=True, exist_ok=True)

### 0.1.2 Herunterladen und Entpacken der Daten zur Anzahl der Hageltage und zur Hagelkorngrösse von ``https://data.geo.admin.ch/``

In [3]:
haildays_file = Path("haildays.zip")
hailsize_file = Path("hailsize.zip")
raw_data_path = Path("./data/raw")
processed_data_path = Path("./data/processed")
out_hail_data_file = "haildata_per_month.csv"
out_hail_filtered_data_file = "filtered_haildata_per_month.csv"

In [4]:
# Erzeugen der Datenpfade, ein Ordner für Rohdaten und einen für prozessierte
create_directory(raw_data_path)
create_directory(processed_data_path)

In [5]:
! wget -O {haildays_file} https://data.geo.admin.ch/ch.meteoschweiz.klima/hageltage/Swiss-hail-climatology_haildays.zip
! wget -O {hailsize_file} https://data.geo.admin.ch/ch.meteoschweiz.klima/hagelkorngroessen/Swiss-hail-climatology_hailsize.zip

--2023-05-21 16:42:37--  https://data.geo.admin.ch/ch.meteoschweiz.klima/hageltage/Swiss-hail-climatology_haildays.zip
Resolving data.geo.admin.ch (data.geo.admin.ch)... 18.165.183.96, 18.165.183.9, 18.165.183.109, ...
Connecting to data.geo.admin.ch (data.geo.admin.ch)|18.165.183.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5853652 (5.6M) [binary/octet-stream]
Saving to: ‘haildays.zip’

haildays.zip        100%[===================>]   5.58M  23.5MB/s    in 0.2s    

2023-05-21 16:42:38 (23.5 MB/s) - ‘haildays.zip’ saved [5853652/5853652]

--2023-05-21 16:42:38--  https://data.geo.admin.ch/ch.meteoschweiz.klima/hagelkorngroessen/Swiss-hail-climatology_hailsize.zip
Resolving data.geo.admin.ch (data.geo.admin.ch)... 18.165.183.19, 18.165.183.9, 18.165.183.109, ...
Connecting to data.geo.admin.ch (data.geo.admin.ch)|18.165.183.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7669619 (7.3M) [binary/octet-stream]
Saving to: ‘hailsi

In [6]:
# Entpacken der heruntergeladenen Hageldaten
unzip_data(haildays_file, raw_data_path)
unzip_data(hailsize_file, raw_data_path)

### Entfernen der zusätzlich heruntergeladenen Daten, die zur Analyse nicht benötigt werden (inklusive der Zip Archive)

In [7]:
# es werden nur die monatlichen Hageldaten (Anzahl als auch Hagelkorngrösse) benötigt
haildays_per_month_file = "haildaysM_ch01r.swiss.lv95_20020401000000_20220930000000.nc"
hailsize_per_month_file = "hailsizeM_ch01r.swiss.lv95_20020401000000_20220930000000.nc"

In [8]:
# Erzeugen einer Liste mit allen zusätzlichen Dateien (ausser der beiden oben definierten)
files_to_delete = [
    str(x) for x in raw_data_path.iterdir() 
    if x.is_file() and x.name not in [haildays_per_month_file, hailsize_per_month_file]
]

# Löschen der zusätzlich mit heruntergeladenen Daten, welche nicht für dieses Projekt benötigt werden
for file in files_to_delete:
    Path(file).unlink()
    
# Entfernen der Zip Archive
haildays_file.unlink()
hailsize_file.unlink()

<a id="cleaning"></a>
## 0.2 Bereinigung der Daten

### 0.2.1 Installation der benötigeten Pakete

In [9]:
!pip install --upgrade pip

In [10]:
!pip install xarray
!pip install netcdf4 h5netcdf scipy pydap zarr fsspec cftime rasterio cfgrib pooch
!pip install pyaxis
!pip install numpy pandas

In [11]:
from pathlib import Path
import xarray as xr
from pyaxis import pyaxis
import pandas as pd
import numpy as np

### 0.2.2 Prozessierung der Hageldaten

In [12]:
haildays_ds = xr.open_dataset(raw_data_path / haildays_per_month_file)
hailsize_ds = xr.open_dataset(raw_data_path / hailsize_per_month_file)

df_haildays = haildays_ds.to_dataframe()
df_hailsize = hailsize_ds.to_dataframe()

/opt/conda/lib/python3.10/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [13]:
df_haildays

lat        lon  swiss_coordinates haildays
time       chy     chx                                                      
2002-04-01 1065500 2480500  45.730684   5.903697                  0      NaT
                   2481500  45.730859   5.916539                  0      NaT
                   2482500  45.731033   5.929380                  0      NaT
                   2483500  45.731205   5.942221                  0      NaT
                   2484500  45.731376   5.955063                  0      NaT
...                               ...        ...                ...      ...
2022-09-01 1303500 2840500  47.837685  10.651764                  0      NaT
                   2841500  47.837316  10.665111                  0      NaT
                   2842500  47.836946  10.678456                  0      NaT
                   2843500  47.836574  10.691802                  0      NaT
                   2844500  47.836200  10.705148                  0      NaT

[10991610 rows x 4 columns]

In [14]:
df_hailsize

lat        lon  swiss_coordinates  hailsize
time       chy     chx                                                       
2002-04-01 1065500 2480500  45.730684   5.903697                  0       NaN
                   2481500  45.730859   5.916539                  0       NaN
                   2482500  45.731033   5.929380                  0       NaN
                   2483500  45.731205   5.942221                  0       NaN
                   2484500  45.731376   5.955063                  0       NaN
...                               ...        ...                ...       ...
2022-09-01 1303500 2840500  47.837685  10.651764                  0       NaN
                   2841500  47.837316  10.665111                  0       NaN
                   2842500  47.836946  10.678456                  0       NaN
                   2843500  47.836574  10.691802                  0       NaN
                   2844500  47.836200  10.705148                  0       NaN

[10991610 rows x 4 columns]

#### Zusammenfügen der Daten zur Anzahl der Hageltage im Monat und zur Hagelkorngrösse über Multiindex

In [15]:
df_hail = df_haildays.merge(df_hailsize["hailsize"], how="inner", left_index=True, right_index=True)
df_hail

lat        lon  swiss_coordinates haildays  \
time       chy     chx                                                         
2002-04-01 1065500 2480500  45.730684   5.903697                  0      NaT   
                   2481500  45.730859   5.916539                  0      NaT   
                   2482500  45.731033   5.929380                  0      NaT   
                   2483500  45.731205   5.942221                  0      NaT   
                   2484500  45.731376   5.955063                  0      NaT   
...                               ...        ...                ...      ...   
2022-09-01 1303500 2840500  47.837685  10.651764                  0      NaT   
                   2841500  47.837316  10.665111                  0      NaT   
                   2842500  47.836946  10.678456                  0      NaT   
                   2843500  47.836574  10.691802                  0      NaT   
                   2844500  47.836200  10.705148                  0      NaT   

                            hailsize  
time       chy     chx                
2002-04-01 1065500 2480500       NaN  
                   2481500       NaN  
                   2482500       NaN  
                   2483500       NaN  
                   2484500       NaN  
...                              ...  
2022-09-01 1303500 2840500       NaN  
                   2841500       NaN  
                   2842500       NaN  
                   2843500       NaN  
                   2844500       NaN  

[10991610 rows x 5 columns]

#### Entfernen der Datenpunlte ohne Werte für `hailsize` oder `haildays` (keine Messung vorhanden)

In [16]:
df_hail = df_hail[~df_hail.haildays.isna()]
df_hail = df_hail[~df_hail.hailsize.isna()]
df_hail

lat       lon  swiss_coordinates haildays  \
time       chy     chx                                                        
2002-04-01 1182500 2634500  46.792770  7.890503                  0   0 days   
           1187500 2610500  46.838557  7.576278                  0   0 days   
           1192500 2609500  46.883549  7.563273                  0   1 days   
                   2610500  46.883534  7.576393                  0   1 days   
           1203500 2660500  46.979808  8.233808                  0   0 days   
...                               ...       ...                ...      ...   
2022-09-01 1294500 2684500  47.795574  8.566370                  0   0 days   
                   2685500  47.795444  8.579714                  0   0 days   
                   2686500  47.795312  8.593059                  0   0 days   
                   2687500  47.795179  8.606403                  0   0 days   
                   2690500  47.794771  8.646434                  0   0 days   

                            hailsize  
time       chy     chx                
2002-04-01 1182500 2634500       2.0  
           1187500 2610500       2.5  
           1192500 2609500       4.0  
                   2610500       4.0  
           1203500 2660500       2.0  
...                              ...  
2022-09-01 1294500 2684500       0.0  
                   2685500       0.0  
                   2686500       0.0  
                   2687500       0.0  
                   2690500       0.0  

[834005 rows x 5 columns]

#### Entfernen des `chy` und `chx` Level vom Multiindex (werden nicht benötigt, da latitude und longitude Daten vorhanden)

In [17]:
df_hail.index = df_hail.index.droplevel([1, 2])
df_hail

,lat,lon,swiss_coordinates,haildays,hailsize
time,,,,,
2002-04-01,46.792770,7.890503,0,0 days,2.0
2002-04-01,46.838557,7.576278,0,0 days,2.5
2002-04-01,46.883549,7.563273,0,1 days,4.0
2002-04-01,46.883534,7.576393,0,1 days,4.0
2002-04-01,46.979808,8.233808,0,0 days,2.0
...,...,...,...,...,...
2022-09-01,47.795574,8.566370,0,0 days,0.0
2022-09-01,47.795444,8.579714,0,0 days,0.0
2022-09-01,47.795312,8.593059,0,0 days,0.0


#### Konvertierung der Spalte `haildays` zu Integer

In [18]:
df_hail["haildays"] = df_hail["haildays"].apply(lambda x: x.days)

<a id="enrichment"></a>
## 0.3 Anreicherung der Daten

#### get `country`, `state` and `district` from `lon` and `lat` data via reverse geocoding

In [19]:
! pip install reverse_geocoder

In [20]:
# offline reverse geocoding
import reverse_geocoder as rg

In [21]:
coordinates = [(x, y) for x, y in zip(df_hail["lat"], df_hail["lon"])]
coordinates[0:10]

[(46.79276983427726, 7.89050251215368),
 (46.83855727394842, 7.576277711422013),
 (46.8835488838835, 7.563273230080763),
 (46.883533848175176, 7.576392784422012),
 (46.97980750155872, 8.233807894747848),
 (47.123720558995366, 8.235943565721179),
 (47.13280560839907, 8.222897425223264),
 (45.99131559201031, 8.993820151171182),
 (45.99113660467614, 9.006722672539098),
 (45.99095613810782, 9.019625105617015)]

In [22]:
reverse_geocoding_results = rg.search(tuple(coordinates))

Loading formatted geocoded file...


In [23]:
reverse_geocoding_results[0:3]

[{'lat': '46.81667',
  'lon': '7.85',
  'name': 'Wald',
  'admin1': 'Bern',
  'admin2': 'Emmental District',
  'cc': 'CH'},
 {'lat': '46.8501',
  'lon': '7.57748',
  'name': 'Niederwichtrach',
  'admin1': 'Bern',
  'admin2': 'Bern-Mittelland District',
  'cc': 'CH'},
 {'lat': '46.87298',
  'lon': '7.561',
  'name': 'Munsingen',
  'admin1': 'Bern',
  'admin2': 'Bern-Mittelland District',
  'cc': 'CH'}]

In [24]:
df_hail["canton"] = np.array([x["admin1"] for x in reverse_geocoding_results])
df_hail["district"] = np.array([x["admin2"] for x in reverse_geocoding_results])
df_hail["country"] = np.array([x["cc"] for x in reverse_geocoding_results])
df_hail = df_hail.reset_index()
df_hail.head()

,time,lat,lon,swiss_coordinates,haildays,hailsize,canton,district,country
0,2002-04-01,46.792770,7.890503,0,0,2.0,Bern,Emmental District,CH
1,2002-04-01,46.838557,7.576278,0,0,2.5,Bern,Bern-Mittelland District,CH
2,2002-04-01,46.883549,7.563273,0,1,4.0,Bern,Bern-Mittelland District,CH
3,2002-04-01,46.883534,7.576393,0,1,4.0,Bern,Bern-Mittelland District,CH
4,2002-04-01,46.979808,8.233808,0,0,2.0,Obwalden,Obwalden,CH


#### Anwendung eines Kanton Mappings und Entfernen der Datenpunkte ausserhalb der Schweiz

In [25]:
df_hail["country"].unique()

array(['CH', 'IT', 'FR', 'AT', 'DE', 'LI'], dtype=object)

In [26]:
canton_mapping = {
    "Zurich": "ZH",
    "Bern": "BE",
    "Lucerne": "LU",
    "Uri": "UR",
    "Schwyz": "SZ",
    "Obwalden": "OW",
    "Nidwalden": "NW",
    "Glarus": "GL",
    "Zug": "ZG",
    "Fribourg": "FR",
    "Solothurn": "SO",
    "Basel-City": "BS",
    "Basel-Landschaft": "BL",
    "Schaffhausen": "SH",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "Saint Gallen": "SG",
    "Grisons": "GR",
    "Aargau": "AG",
    "Thurgau": "TG",
    "Ticino": "TI",
    "Vaud": "VD",
    "Valais": "VS",
    "Neuchatel": "NE",
    "Geneva": "GE",
    "Jura": "JU",
}

In [27]:
set(df_hail["canton"].unique()) - set(canton_mapping.keys())

{'Alsace',
 'Aosta Valley',
 'Baden-Wuerttemberg',
 'Balzers',
 'Franche-Comte',
 'Gamprin',
 'Lombardy',
 'Piedmont',
 'Rhone-Alpes',
 'Ruggell',
 'Trentino-Alto Adige',
 'Tyrol',
 'Vorarlberg'}

In [28]:
df_hail = df_hail[df_hail["canton"].isin(canton_mapping.keys())]
df_hail = df_hail.replace({"canton": canton_mapping})
df_hail["country"].unique()

array(['CH'], dtype=object)

#### Hinzufügen von separaten Spalten für Monat und Jahr

In [29]:
df_hail["year"] = df_hail["time"].apply(lambda x: x.year)
df_hail["month"] = df_hail["time"].apply(lambda x: x.month)

#### Entfernen von unnötigen Spalten (`swiss_coordinates`, `district`, `country`)

In [30]:
df_hail = df_hail.drop(["swiss_coordinates", "district", "country"], axis=1)

#### Umordnung der Spalten in schönerer Reihenfolge

In [31]:
columns = ["time", "year", "month", "lat", "lon", "canton", "haildays", "hailsize"]
df_hail = df_hail[columns]
df_hail.head()

,time,year,month,lat,lon,canton,haildays,hailsize
0,2002-04-01,2002,4,46.792770,7.890503,BE,0,2.0
1,2002-04-01,2002,4,46.838557,7.576278,BE,0,2.5
2,2002-04-01,2002,4,46.883549,7.563273,BE,1,4.0
3,2002-04-01,2002,4,46.883534,7.576393,BE,1,4.0
4,2002-04-01,2002,4,46.979808,8.233808,OW,0,2.0


<a id="dump"></a>
## 0.4 Speichern der Daten

#### Speichern der vorprozessierten Daten

In [32]:
df_hail.to_csv(processed_data_path / out_hail_data_file, sep=",", index=False)

#### Speichern der vorprozessierten Daten mit zusätzlichen FIlter nach Hagelkorngrösse

In [33]:
# minimale Hagelkorngrösse in cm
min_hail_size = 2.0

df_hail_filtered = df_hail[
    (df_hail["haildays"] > 0.0)
    & (df_hail["hailsize"] >= min_hail_size)
]
df_hail_filtered.head()

,time,year,month,lat,lon,canton,haildays,hailsize
2,2002-04-01,2002,4,46.883549,7.563273,BE,1,4.0
3,2002-04-01,2002,4,46.883534,7.576393,BE,1,4.0
11,2002-05-01,2002,5,46.082633,8.892995,TI,1,4.5
12,2002-05-01,2002,5,46.082465,8.905919,TI,1,4.5
13,2002-05-01,2002,5,46.091627,8.893235,TI,1,2.0


In [34]:
df_hail_filtered.to_csv(processed_data_path / out_hail_filtered_data_file, sep=",", index=False)